**First International Summer School in Data Science for Mobility (DSM)**, 3-7 October 2022, Santorini, Greece

<img width=200, src="http://master-school.isti.cnr.it/wp-content/uploads/2022/05/logo-Summer-School-Mobility-2022.png"/>

Author: [Luca Pappalardo](https://twitter.com/lucpappalard)

# Exercises 2.x

## Exercise 2.1
Download Cell Towers data about France from [OpenCellID](https://opencellid.org/). Then, create and visualize (in folium) a Voronoi tessellation of GSM towers in Lyon, France.
- Use the [geovoronoi](https://pypi.org/project/geovoronoi/) package to create the Voronoi tessellation
- Plot the tessellation with folium (Use just a random sample if the towers if they are too many to be visualized and/or to compute the tessellation) 

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
FRANCE = 'data/208.csv.gz'

In [ ]:
columns = ['Radio', 'MCC', 'MNC', 'LAC/TAC/NID', 'CID', 'Longitude', 'Latitude', 
                                 'Range', 'Samples', 'Changeable', 'Created', 'Updated']
df = pd.read_csv(FRANCE, header=None).drop([5, 13], axis=1)
df.columns = columns
print(len(df))
df.head()

In [ ]:
df_gsm = df[df['Radio'] == 'GSM']
df_gsm.head()

Let's transform it into a `GeoDataFrame`

In [ ]:
%%time
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs='EPSG:4326')
gdf.head()

Let's download the shape of Lyon, France 

In [ ]:
import osmnx as ox

In [ ]:
lyon_gdf = ox.geocode_to_gdf('Lyon, France')
ax = ox.project_gdf(lyon_gdf).plot()
_ = ax.axis('off')

In [ ]:
lyon_poly = lyon_gdf['geometry'].iloc[0]
lyon_poly

In [ ]:
%%time
gdf_lyon = gdf.sjoin(lyon_gdf)
gdf_lyon.head()

In [ ]:
from geovoronoi import voronoi_regions_from_coords, points_to_coords
from geovoronoi import voronoi_regions_from_coords
from scipy.spatial import Voronoi,voronoi_plot_2d

In [ ]:
def to_GeoDataFrame(region_polys):
    name=[]
    for i in range(1, len(region_polys) + 1):
        name.append('cell ' + str(i))
    gdf = gpd.GeoDataFrame(columns=['name','geometry'], crs={'init': 'epsg:4326'})
    gdf['name'] = name
    for index, row in gdf.iterrows():
        gdf.at[index, 'geometry'] = region_polys[index]
    return gdf


def get_voronoi_tessellation(poly_ch, points):
    vor = Voronoi(points, qhull_options='Qbb Qc Qx')
    region_polys, region_pts = voronoi_regions_from_coords(points, poly_ch)
    tess_voronoi = to_GeoDataFrame(region_polys)
    return tess_voronoi

In [ ]:
vor_centroids = gdf_lyon[['Longitude', 'Latitude']].to_numpy()
vor_centroids

In [ ]:
%%time
tess_voronoi_lyon = get_voronoi_tessellation(lyon_poly, vor_centroids)
tess_voronoi_lyon.head()

In [ ]:
import folium

In [ ]:
map_f = folium.Map(
    location=[lyon_poly.centroid.y, lyon_poly.centroid.x],
    tiles="cartodbpositron",
    zoom_start=12,
)

# city 
sim_geo = gpd.GeoSeries(lyon_poly)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
                       style_function=lambda x: {'fillColor': 'orange', 'color': 'blue', 'weight': 1})
geo_j.add_to(map_f)


# voronoi
sim_geo = gpd.GeoSeries(tess_voronoi_lyon['geometry'])
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
                       style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 0.5})
geo_j.add_to(map_f)

map_f

## Exercise 2.2
Use OSMnx to download the shapes of all cinemas in Rome.
- Plot in folium the shape of the municipality of Rome
- Plot the smallest polygon that contains all cinemas in Rome
- Plot the cinemas in folium with a pop-up showing their name
- Compute the cinemas with the lowest and highest average distance to the others, and draw a circle (of radius 100m)

In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from scipy.spatial import Voronoi
from geovoronoi import voronoi_regions_from_coords

In [ ]:
df_regions = pd.read_json('https://raw.githubusercontent.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel/master/italy_regions.json')
df_regions.head()

In [ ]:
geo_df = pd.read_json('https://raw.githubusercontent.com/MatteoHenryChinaski/Comuni-Italiani-2018-Sql-Json-excel/master/italy_geo.json')
geo_df["lng"] = pd.to_numeric(geo_df["lng"])
print(len(geo_df))
geo_df.head()

In [ ]:
capitals = ["Aosta", "Torino", "Genova", "Milano", "Trento", "Venezia", "Trieste", "Bologna", "Firenze", "Perugia", 
"Ancona", "Roma", "L'Aquila", "Campobasso", "Napoli", "Bari", "Potenza", "Catanzaro", "Palermo", "Cagliari"]
capitals_df = geo_df[geo_df['comune'].isin(capitals)]
capitals_df

In [ ]:
capitals_gdf = gpd.GeoDataFrame(geo_df[geo_df['comune'].isin(capitals)], geometry=gpd.points_from_xy(capitals_df['lng'], capitals_df['lat']), crs='EPSG:4326')
capitals_gdf

In [ ]:
italy_gdf = ox.geocode_to_gdf('Italy')
italy_poly = italy_gdf['geometry'].iloc[0]
italy_poly

In [ ]:
def to_GeoDataFrame(region_polys):
    name=[]
    for i in range(1, len(region_polys) + 1):
        name.append('cell ' + str(i))
    gdf = gpd.GeoDataFrame(columns=['name','geometry'], crs={'init': 'epsg:4326'})
    gdf['name'] = name
    for index, row in gdf.iterrows():
        gdf.at[index, 'geometry'] = region_polys[index]
    return gdf


def get_voronoi_tessellation(poly_ch, points):
    vor = Voronoi(points, qhull_options='Qbb Qc Qx')
    region_polys, region_pts = voronoi_regions_from_coords(points, poly_ch)
    tess_voronoi = to_GeoDataFrame(region_polys)
    return tess_voronoi

In [ ]:
vor_centroids = capitals_gdf[['lng', 'lat']].to_numpy()

In [ ]:
%%time
tess_voronoi = get_voronoi_tessellation(italy_poly, vor_centroids)
print(len(tess_voronoi))
tess_voronoi.head()

In [ ]:
import folium

In [ ]:
map_f = folium.Map(
    location=[italy_poly.centroid.y, italy_poly.centroid.x],
    tiles="cartodbpositron",
    zoom_start=5,
)

# city 
sim_geo = gpd.GeoSeries(italy_poly)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
                       style_function=lambda x: {'fillColor': 'orange', 'color': 'blue', 'weight': 1})
geo_j.add_to(map_f)


# voronoi
sim_geo = gpd.GeoSeries(tess_voronoi['geometry'])
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j,
                       style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 0.5})
geo_j.add_to(map_f)

map_f

## Exercise 2.3
Download from figshare this [flows dataset](https://figshare.com/collections/Inter-urban_interactions_of_mobility_via_cellular_position_tracking_in_the_southeast_Songliao_Basin_Northeast_China/4226183), create a tessellation and a `FlowDataFrame`; plot them together using skmob.

In [ ]:
import requests, zipfile, json, io
from shapely.geometry import Point

In [ ]:
import pandas as pd
import geopandas as gpd
import skmob

In [ ]:
dataset_links = {
'positions' : 'https://figshare.com/ndownloader/files/14005292',
'flows' : 'https://figshare.com/ndownloader/files/14884442',
}

Download the tessellation:

In [ ]:
r = requests.get(dataset_links['positions'], stream=True)
print(r.text.replace('\r', '\n'), file=open('positions.csv','w'))

In [ ]:
positions_df = pd.read_csv('positions.csv')
positions_df.head()

In [ ]:
gdf = gpd.GeoDataFrame(positions_df, 
                       geometry=gpd.points_from_xy(positions_df['Longitude'], positions_df['Latitude'])).drop(['Longitude', 'Latitude'], axis=1).rename(columns={'Location': 'tile_ID'})
gdf.head()

Download the flows:

In [ ]:
r = requests.get(dataset_links['flows'], stream=True)
print(r.text, file=open('flows.csv','w'))

In [ ]:
fdf = skmob.FlowDataFrame(pd.read_csv('flows.csv'), origin='Origin', destination='Destination', flow='Weight', tile_id='tile_ID', tessellation=gdf)
fdf.head()

In [ ]:
fdf.plot_flows(min_flow=500)